In [1]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
# from huggingface_hub import notebook_login

# notebook_login()


In [3]:

!nvidia-smi

Thu Aug 31 14:26:32 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:AF:00.0 Off |                    0 |
| N/A   66C    P0              32W /  70W |      2MiB / 15360MiB |      6%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
PATH = "/root/.cache/huggingface/hub/models--meta-llama--Llama-2-7b-chat-hf"
model_name = "FlagAlpha/Llama2-Chinese-7b-Chat"
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name,
                                          local_files_only=True
                                          # use_auth_token=True,
                                         )



In [5]:
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             local_files_only=True,
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             temperature=0.2, # must be strictly positive float
                                             do_sample=True,
                                             # use_auth_token=True,
                                            #  load_in_8bit=True,
                                            #  load_in_4bit=True
                                             )

2023-08-31 14:26:37.146732: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-31 14:26:38.034124: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [6]:
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                # return_full_text=True,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [7]:
from langchain.llms import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=pipe)

In [8]:
llm("你好嗎?")

/usr/local/lib/python3.10/site-packages/transformers/generation/utils.py:1411: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


'\n[你好嗎？] 你好嗎？这是一个问候语，意思是你好吗？这是一个常用的开场白，可以用于询问对方的情况或讨论话题。\n\n[你好嗎？] 你好嗎？这是一个问候语，意思是你好吗？这是一个常用的开场白，可以用于询问对方的情况或讨论话题。\n\n[你好嗎？] 你好嗎？这是一个问候语，意思是你好吗？这是一个常用的开场白，可以用于询问对方的情况或讨论话题。\n\n[你好嗎？] 你好嗎？这是一个问候语，意思是你好吗？这是一个常用的开场白，可以用于询问对方的情况或讨论话题。\n\n[你好嗎？] 你好嗎？这是一个问候语，意思是你好吗？这是一个常用的开场白，可以用于询问对方的情况或讨论话题。\n\n[你好嗎？] 你好嗎？这是一个问候语，意思是你好吗？这是一个常用的开场白，可以用于询问对方的情况或讨论话题。\n\n[你好嗎？] 你好嗎？这是一个问候语，意思是你好吗？这是一个常用的开场白，可以用于询问对方的情况或讨论话题。\n\n[你好嗎？] 你好嗎？这是一个问候'

In [9]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Thu Aug 31 14:27:22 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T

In [12]:
# !set PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:6144

In [14]:
# !set TOKENIZERS_PARALLELISM=true

In [19]:
template = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。

  {context}

  问题: {question}
  答案:"""

In [20]:
from langchain import PromptTemplate

prompt_template = PromptTemplate.from_template(
    template
)

In [21]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt_template)

In [22]:
%%time
chain.invoke(
    {
        # "output_language" : "Tranditional Chinese",
        # "output_language" : "Simplified Chinese",
        # "output_language" : "English",
        "context" : "我是伊森，我是一名软体工程师。",
        "question": "我从事什么行业?",
        # "eos_token_id"=tokenizer.eos_token_id,
        # "pad_token_id"=tokenizer.pad_token_id
    }
)

CPU times: user 899 ms, sys: 282 ms, total: 1.18 s
Wall time: 1.18 s


{'context': '我是伊森，我是一名软体工程师。',
 'question': '我从事什么行业?',
 'text': ' 我从事软体工程师行业。\n'}

In [45]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Thu Aug 31 15:00:17 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T

# Q1

In [23]:
question1= "如何设定判定逻辑?"
content1 = """
  选择是否设定附加规则 , 限制 SN长度或厂家代码范围等 , 设定完毕后 , 点击【保存】 , 
完成该产品序列号判定逻辑设定 .2.2 检验逻辑设定   
逻辑判定设置完成后 , 厂家保修管理 后台->基础设定 ->判定逻辑 , 点击进入逻辑判定
页面, 最下方输入 PPID, 点击【解析】检验逻辑设定 ;如下图, 可依规则自动解析出厂
时间等信息 . 
 
3. 保修类别  
厂家保修管理 后台->基础设定 ->保修类别，点击【新增设定】 , 设定需要为产品配置
的保修类别 ,填写服务类型、时效值及单位 , 点击【保存】完成 ; 
 
 
4. 保修约定  
当产品保修及保修卡无法满足保修需求时 , 可添加保修约定 .基础设定  
1. 判定命名  
厂家保修管理 后台->基础设定 ->判定命名 ,依用户习惯 , 自主设定相关栏位名称 , 设定
后, 系统相关栏位均以设定名称显示 . 
 
2. 判定逻辑  
设定PPID/SN的逻辑规则 , 用于解析保修起点日期等信息 . 
2.1 添加/编辑判定逻辑  
厂家保修管理 后台->基础设定 ->判定逻辑 , 点击【添加】按钮 , 添加判定逻辑 ;选定判
定逻辑, 点击对该判定逻辑进行编辑 ; 
 
填写解码规则名称、说明、 SN样例；点击【编辑】按钮 进行应用 ; 
 
填写 PPID/SN子串区段 , 依照子串区段 的代表意义 , 可自定义规则  
如下图中点击【月份 规则自定义】 ，  进入自定义模块后 , 点击【新增码值】 , 可自定义
月份解析规则 , 例如 , 可依 PPID/SN规则设定 A代表 10月, B代表 11月份 , C代表 12月
份; 
【添加一项自定义规则】可添加 PPID/SN子串逻辑自定义规则 .
"""

In [11]:
%%time
out1 = llm(prompt1)

CPU times: user 14 s, sys: 5.62 s, total: 19.6 s
Wall time: 19.6 s


In [12]:
out1

'  1. 在厂家保修管理后台中，点击“基础设定”，然后点击“判定逻辑”。\n  2. 在“判定逻辑”页面中，选择要设定的判定逻辑，然后点击“编辑”按钮。\n  3. 在“编辑”页面中，填写解码规则名称、说明、 SN样例，然后点击“编辑”按钮进行应用。\n  4. 在“编辑”页面中，可以自定义规则，如月份解析规则，可依 PPID/SN规则设定 A代表 10月，B代表 11月份，C代表 12月份。\n'

In [43]:
%%time
out1 = chain.invoke(
    {
        # "output_language" : "Tranditional Chinese",
        # "output_language" : "Simplified Chinese",
        # "output_language" : "English",
        "context" : content1,
        "question": question1,
        # "eos_token_id"=tokenizer.eos_token_id,
        # "pad_token_id"=tokenizer.pad_token_id
    }
)

/usr/local/lib/python3.10/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


CPU times: user 11.2 s, sys: 3.99 s, total: 15.2 s
Wall time: 15.2 s


In [44]:
out1['text']

' 设定PPID/SN的逻辑规则，用于解析保修起点日期等信息。可以通过添加判定逻辑，选定判定逻辑，填写解码规则名称、说明、 SN样例，点击编辑按钮进行应用。如果需要自定义规则，可以在自定义模块中新增码值，点击新增码值，可自定义月份解析规则，例如，可依 PPID/SN规则设定 A代表 10月，B代表 11月份，C代表 12月份。\n'

A1: 在厂家保修管理后台中的基础设定中进行判定逻辑的添加和编辑。首先，在厂家保修管理后台中，点击基础设定，然后选择判定逻辑。接着，点击添加按钮，填写解码规则名称、说明、SN样例，然后点击编辑按钮进行应用。最后，填写PPID/SN子串区段，依照子串区段的代表意义，可自定义规则。例如，可以依PPID/SN规则设定A代表10月，B代表11月份，C代表12月份。

## Ans2: 在哪里可以设定机种?

In [29]:
question2= "在哪里可以设定机种?"
content2 = """
  选择是否设定附加规则 , 限制 SN长度或厂家代码范围等 , 设定完毕后 , 点击【保存】 , 
完成该产品序列号判定逻辑设定 .选定不良代码 , 点击进入客户绑定页面 ; 勾选客户 , 点击【保存】完成不良判定代码
与客户的绑定 ; 
 
 
保修设定  
1. 定义产品 (整机料号信息 ) 
单笔或批量将产品信息导入系统 ,包含机种、厂家料号、客户料号、供应商料号、替代
料号等信息  
1.1 机种管理  
若产品涉及机种 , 需先将机种信息新增至系统 , 以便后续选择 . 
a. 单笔新增机种  
厂家保修管理 后台 ->保修设定 ->定义产品 , 点击【机种管理】按钮 , 点击【新增】填写
机种名称 , 点击【保存】完成 ;1.9 产品与附件绑定  
本操作步骤 , 需要在本章 2. 定义附件  操作完成后进行 . 
厂家保修管理 后台 ->保修设定 ->定义产品 , 选定产品 , 点击按钮 , 点击【绑定附件】并勾
选对应附 件、填写数量 , 点击【绑定】完成 ; 
 
 
2. 定义部件 （备品料号信息）  
若部件中涉及机种管理 , 亦需先新增或上传机种 , 而后操作步骤同定义产品  
2.1 新增部件机种  
点击【机种管理】按钮 , 点击【新增】单笔新增机种 ; 点击【上传机种】下载模板 , 批
量新增机种 ;  
2.2 新增部件  
点击【新增部件】单笔新增部件 ; 点击【上传部件】下拉框 , 下载模板批量新增部件 ; 
2.3 新增部件客户料号 ; 
点击【料号对照】添加客户料号 , 选定产品 , 点击单笔添加 ; 点击【料号对照】下载模
板, 批量上传料号 ; 
2.3 新增供应商料号  
点击【供应商料号】 , 选定产品 , 点击按钮 , 点击【添加供应商】 , 添加供应商及供应
商料号 , 【保存】完成 .
"""

In [30]:
%%time
out2 = chain.invoke(
    {
        # "output_language" : "Tranditional Chinese",
        # "output_language" : "Simplified Chinese",
        # "output_language" : "English",
        "context" : content2,
        "question": question2,
        # "eos_token_id"=tokenizer.eos_token_id,
        # "pad_token_id"=tokenizer.pad_token_id
    }
)

CPU times: user 3.14 s, sys: 1.35 s, total: 4.49 s
Wall time: 4.49 s


In [31]:
out2['text']

' 在厂家保修管理 后台 -> 保修设定 -> 定义产品 -> 机种管理 -> 单笔新增机种。\n'

ANS2: 在厂家保修管理后台中的保修设定中进行机种的设定。首先，在厂家保修管理后台中，点击保修设定，然后选择定义产品。接着，点击机种管理按钮，可以进行单笔新增机种或批量新增机种。填写机种名称后，点击保存即可完成机种的设定。

Q3

In [34]:
question3= "平台可以自动核准保修申请吗?"
content3 = """
  点击【上传附件】下拉框 , 选择【下载模板】填写附件相关信息 , 【批量上传】将编辑好
的文档上传 , 完成批量新增附件 . 
 
保修作业  
1. 保修申请 （手动核准） 
若厂家在客户清单中 , 对应客户已设置【自动核准】功能 ; 客户进行保修申请后 , 平台
依保修条件进行自动核准并分配 ware no.; 若未设定自动核准 , 即可通过手动核准进行
作业 ; 手动核准作业如下 : 
客户进行保修申请后 , 厂家后台保修申请方有数据 ; 
厂家保修管理 后台 ->保修作业 ->保修申请 , 选择对应送修项目 , 点击按钮进入送修信息 ; 
 
点击可查看客户信息 , 点击【接受】进行核准 , 点击【拒绝】进行驳回 ， 需填写拒绝
原因; 当客户送修系统判定不可接受时 (即不符合厂家保修设定 ), 点击可变更核准状
态, 选择保修设定后接受或者拒绝 ;3. 单号提交  
厂家保修管理 后台->保修作业 ->单号提交 , 点击按钮, 勾选对应栏位 , 点击【确定】
可移动或合并 Ware no.分组(同客户同申请人可移动至同一组内 ); 点击提交单号 , 
ware no.即生成. 
 
4. 进度追踪  
4.1 查看保修进度  
厂家保修管理 后台->保修作业 ->进度追踪 , 点击【保修单号】 , 点击进入送修信息 ; 可
查看保修及物流进度 ; 【查看报告】 查看服务中心的维修报告 ; 点击可查看该笔 SN的
产品详情、保修详情、物流资讯和服务进度 .2.2 检验逻辑设定   
逻辑判定设置完成后 , 厂家保修管理 后台->基础设定 ->判定逻辑 , 点击进入逻辑判定
页面, 最下方输入 PPID, 点击【解析】检验逻辑设定 ;如下图, 可依规则自动解析出厂
时间等信息 . 
 
3. 保修类别  
厂家保修管理 后台->基础设定 ->保修类别，点击【新增设定】 , 设定需要为产品配置
的保修类别 ,填写服务类型、时效值及单位 , 点击【保存】完成 ; 
 
 
4. 保修约定  
当产品保修及保修卡无法满足保修需求时 , 可添加保修约定 .
"""

In [35]:
%%time
out3 = chain.invoke(
    {
        # "output_language" : "Tranditional Chinese",
        # "output_language" : "Simplified Chinese",
        # "output_language" : "English",
        "context" : content3,
        "question": question3,
        # "eos_token_id"=tokenizer.eos_token_id,
        # "pad_token_id"=tokenizer.pad_token_id
    }
)

CPU times: user 4.7 s, sys: 1.73 s, total: 6.44 s
Wall time: 6.45 s


In [36]:
out3['text']

' 平台可以自动核准保修申请，但是需要客户在客户清单中设置自动核准功能。如果客户未设置自动核准，则需要手动核准。\n'

ANS3: 可以在客户清单中设置自动核准功能。在客户管理中，可以选择对应的客户，然后勾选自动核准功能。开启自动核准功能后，当客户提交保修申请时，系统会自动进行核准并分配ware no.，无需手动核准。但需要注意的是，自动核准功能需要满足一定的保修条件，否则仍需要手动核准。

## Q4

In [37]:
question4= "自动核准后还需要进行单号提交吗?"
content4 = """
  2. 单号分配  
厂家保修管理 后台->保修作业 ->单号分配 , 勾选送修项目 , 点击【分配 Ware no. 】分
配单号,同客户同申请人的送修清单 , 可一次分配 Ware no.;  
 
勾选送修项目 , 点击【指派 Ware no.】, 系统根据产品型号、退运原因、维修地址、服
务类型等对产品分类并分配  Ware no.;  
备注 : 若客户需使用自主 RMA no.，亦可在【自定单号】添加客户指定 RMA no.点击【上传附件】下拉框 , 选择【下载模板】填写附件相关信息 , 【批量上传】将编辑好
的文档上传 , 完成批量新增附件 . 
 
保修作业  
1. 保修申请 （手动核准） 
若厂家在客户清单中 , 对应客户已设置【自动核准】功能 ; 客户进行保修申请后 , 平台
依保修条件进行自动核准并分配 ware no.; 若未设定自动核准 , 即可通过手动核准进行
作业 ; 手动核准作业如下 : 
客户进行保修申请后 , 厂家后台保修申请方有数据 ; 
厂家保修管理 后台 ->保修作业 ->保修申请 , 选择对应送修项目 , 点击按钮进入送修信息 ; 
 
点击可查看客户信息 , 点击【接受】进行核准 , 点击【拒绝】进行驳回 ， 需填写拒绝
原因; 当客户送修系统判定不可接受时 (即不符合厂家保修设定 ), 点击可变更核准状
态, 选择保修设定后接受或者拒绝 ;3. 单号提交  
厂家保修管理 后台->保修作业 ->单号提交 , 点击按钮, 勾选对应栏位 , 点击【确定】
可移动或合并 Ware no.分组(同客户同申请人可移动至同一组内 ); 点击提交单号 , 
ware no.即生成. 
 
4. 进度追踪  
4.1 查看保修进度  
厂家保修管理 后台->保修作业 ->进度追踪 , 点击【保修单号】 , 点击进入送修信息 ; 可
查看保修及物流进度 ; 【查看报告】 查看服务中心的维修报告 ; 点击可查看该笔 SN的
产品详情、保修详情、物流资讯和服务进度 .
"""

In [38]:
%%time
out4 = chain.invoke(
    {
        # "output_language" : "Tranditional Chinese",
        # "output_language" : "Simplified Chinese",
        # "output_language" : "English",
        "context" : content4,
        "question": question4,
        # "eos_token_id"=tokenizer.eos_token_id,
        # "pad_token_id"=tokenizer.pad_token_id
    }
)

CPU times: user 1.17 s, sys: 583 ms, total: 1.76 s
Wall time: 1.76 s


In [39]:
out4['text']

' 不需要进行单号提交。\n'

ANS4: 关于您的问题，当开启自动核准功能后，系统会自动进行核准并分配ware no.，无需手动核准。因此，在这种情况下，不需要进行单号提交。但需要注意的是，自动核准功能需要满足一定的保修条件，否则仍需要手动核准。

Q5

In [40]:
question5= "怎么查看保修进度?"
content5 = """
  3. 单号提交  
厂家保修管理 后台->保修作业 ->单号提交 , 点击按钮, 勾选对应栏位 , 点击【确定】
可移动或合并 Ware no.分组(同客户同申请人可移动至同一组内 ); 点击提交单号 , 
ware no.即生成. 
 
4. 进度追踪  
4.1 查看保修进度  
厂家保修管理 后台->保修作业 ->进度追踪 , 点击【保修单号】 , 点击进入送修信息 ; 可
查看保修及物流进度 ; 【查看报告】 查看服务中心的维修报告 ; 点击可查看该笔 SN的
产品详情、保修详情、物流资讯和服务进度 .3. 产品保修 （维修服务 , “退运流程”设置完成后设置）  
厂家后台 ->保修设定 ->产品保修 , 选定对应客户 , 点击按钮进入页面 ; 点击【新增】设
置产品保修 ; 
 
 
点击【选择类别】 , 选择维修 /翻新保修约定 , 选择预设置产品所对应的产品类别 , 选
择保修期间服务内容、具体服务流程、保修期间时长等信息 , 填写完成点击【下一
步】; 
备注: 保修期间服务内容 , 为上章基础设定中 【保修类别】中设定之 信息; 【服务流
程】为送修至服务中心 , 本次需选择；  【服务地址 】为送修至厂家 , 不经服务中心 , 
本次不选择 ; 
 
添加产品 PN, 点击【批量添加 PN】， 带出相同类别 PN产品, 勾选后确认 , 即完成产
品保修设定2.2 检验逻辑设定   
逻辑判定设置完成后 , 厂家保修管理 后台->基础设定 ->判定逻辑 , 点击进入逻辑判定
页面, 最下方输入 PPID, 点击【解析】检验逻辑设定 ;如下图, 可依规则自动解析出厂
时间等信息 . 
 
3. 保修类别  
厂家保修管理 后台->基础设定 ->保修类别，点击【新增设定】 , 设定需要为产品配置
的保修类别 ,填写服务类型、时效值及单位 , 点击【保存】完成 ; 
 
 
4. 保修约定  
当产品保修及保修卡无法满足保修需求时 , 可添加保修约定 .
"""

In [41]:
%%time
out5 = chain.invoke(
    {
        # "output_language" : "Tranditional Chinese",
        # "output_language" : "Simplified Chinese",
        # "output_language" : "English",
        "context" : content5,
        "question": question5,
        # "eos_token_id"=tokenizer.eos_token_id,
        # "pad_token_id"=tokenizer.pad_token_id
    }
)

CPU times: user 2.97 s, sys: 1.32 s, total: 4.29 s
Wall time: 4.29 s


In [42]:
out5['text']

' 在厂家后台->保修作业->进度追踪中，点击保修单号，可查看保修及物流进度。\n'

ANS5: 可以在厂家保修管理后台中查看保修进度。在保修作业中，选择进度追踪，然后点击对应的保修单号，即可查看保修及物流进度。同时，您也可以点击查看报告，查看服务中心的维修报告。如果需要查看该笔产品的详情、保修详情、物流资讯和服务进度，可以点击对应的SN进行查看。